## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [1]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [3]:
import pandas as pd
import numpy as np
import numba
import cProfile

recipes = pd.read_csv("D:\Programming\Python\BigDataPT-BDPT-\Lesson3\data\data\\recipes_sample.csv", parse_dates=["submitted"])
reviews = pd.read_csv("D:\Programming\Python\BigDataPT-BDPT-\Lesson3\data\data\\reviews_sample.csv", parse_dates=["date"], index_col=0)

In [3]:
count = 0
for _, row in reviews.iterrows():
    count += row[3]
    # print(_, row[3])
count/reviews.shape[0]

4.410802235271832

In [4]:
tenYear_reviews = reviews[reviews["date"].dt.year == 2010]
count = 0
for _, row in tenYear_reviews.iterrows():
    count += row[3]
    # print(_, row[3])
count/tenYear_reviews.shape[0]

4.4544402182900615

In [9]:
reviews["rating"].mean()

4.410802235271832

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [6]:
def taskA(reviews):
    count = 0
    for _, row in reviews.iterrows():
        count += row[3]
    return count/reviews.shape[0]

def taskB(reviews):
    tenYear_reviews = reviews[reviews["date"].dt.year == 2010]
    count = 0
    for _, row in tenYear_reviews.iterrows():
        count += row[3]
    return count/tenYear_reviews.shape[0]

def taskC(reviews):
    reviews["rating"].mean()

In [7]:
%load_ext line_profiler

In [8]:
%lprun -f taskA taskA(reviews)

Timer unit: 1e-07 s

Total time: 14.0695 s

Could not find file C:\Users\damir\AppData\Local\Temp/ipykernel_11884/2338034132.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1
     2         1         18.0     18.0      0.0
     3    126697  130774453.0   1032.2     92.9
     4    126696    9920584.0     78.3      7.1
     5         1         96.0     96.0      0.0

In [9]:
%lprun -f taskB taskB(reviews)

Timer unit: 1e-07 s

Total time: 1.39642 s

Could not find file C:\Users\damir\AppData\Local\Temp/ipykernel_11884/2338034132.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     7
     8         1     178661.0 178661.0      1.3
     9         1         16.0     16.0      0.0
    10     12095   12809670.0   1059.1     91.7
    11     12094     975746.0     80.7      7.0
    12         1         71.0     71.0      0.0

In [10]:
%lprun -f taskC taskC(reviews)

Timer unit: 1e-07 s

Total time: 0.0009295 s

Could not find file C:\Users\damir\AppData\Local\Temp/ipykernel_11884/2338034132.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    14
    15         1       9295.0   9295.0    100.0

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [11]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [12]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 48.8676 s

Could not find file C:\Users\damir\AppData\Local\Temp/ipykernel_11884/470240673.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1         16.0     16.0      0.0  
     3    126680  134869377.0   1064.6     27.6  
     4    126679   28074503.0    221.6      5.7  
     5    126679    5563176.0     43.9      1.1  
     6   6918689   22680689.0      3.3      4.6  
     7   6792010   31889186.0      4.7      6.5  
     8    174944     942863.0      5.4      0.2  
     9   6792010   36237151.0      5.3      7.4  
    10                                           
    11         1         11.0     11.0      0.0  
    12    126680  132043648.0   1042.3     27.0  
    13    126679   17024392.0    134.4      3.5  
    14    126679    5612034.0     44.3      1.1  
    15   6918689   23899525.0      3.5      4.9  
    16   6792010   49839625.0      7.3     10.2  
    17         1        141.0    141.0      0.0

In [18]:
# @numba.jit
def get_word_reviews_count_improve(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        words = row['review'].split(" ")

        for word in words:
            if word not in word_reviews:
                word_reviews[word] = 1
            else:
                word_reviews[word] += 1
    return word_reviews

In [49]:
@numba.jit
def get_word_reviews_count_improve(df):
    word_reviews = {}
    for row in df['review'].head(5).dropna():
        print(type(row))
#         words = row['review'].split(" ")

#         for word in words:
#             if word not in word_reviews:
#                 word_reviews[word] = 1
#             else:
#                 word_reviews[word] += 1
    return word_reviews

In [47]:
get_word_reviews_count_improve(reviews)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


{}

In [30]:
reviews.head(5)['review'].to_numpy()

array(["Last week whole sides of frozen salmon fillet was on sale in my local supermarket, so I bought tons (okay, only 3, but total weight was over 10 pounds).  This recipe is perfect for salmon fillet, even though it calls for salmon steaks.  I cut up the salmon into individual portions and followed the instructions exactly.  I'm on one of those food combining diets, so I left out the white wine but added just a dash of white wine vinegar instead (just a little bit, not enough to change the taste of the dish).  Super yummy, and leftovers for lunch today (lucky me)!",
       "So simple and so tasty!  I used a yellow capsicum in place of the green because that's what I had on hand.  This came together so fast.  Perfect meal if you don't have a lot of time.  Easy, healthy and tasty.  Thanks Stardustannie!  Made for PAC Fall 2007.",
       'Very nice breakfast HH, easy to make and yummy with fresh hot coffe.  Instead of toast I served on recipe #97694 by Paula G that I had made earlier a

In [32]:
get_word_reviews_count_improve(reviews)

KeyError: 'reviews'

In [15]:
%lprun -f get_word_reviews_count_improve get_word_reviews_count_improve(reviews)

C:\Users\damir\AppData\Local\Temp/ipykernel_11668/1601311374.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_word_reviews_count_improve" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\damir\AppData\Local\Temp/ipykernel_11668/1601311374.py (3)

File "C:\Users\damir\AppData\Local\Temp\ipykernel_11668\1601311374.py", line 3:
<source missing, REPL/exec in use?>

  @numba.jit
C:\Users\damir\AppData\Local\Temp/ipykernel_11668/1601311374.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "get_word_reviews_count_improve" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "C:\Users\damir\AppData\Local\Temp\ipykernel_11668\1601311374.py", line 4:
<source missing, REPL/exec in use?>

  @numba.jit
D:\Programming\Python\BigDataPT-BDPT-\venv\lib\site-packages\numba\core\objec

ValueError: max() arg is an empty sequence

Timer unit: 1e-07 s

Total time: 19.8699 s

Could not find file C:\Users\damir\AppData\Local\Temp/ipykernel_3592/1855540428.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2
     3         1         12.0     12.0      0.0
     4    126680  112893392.0    891.2     56.8
     5    126679   18796458.0    148.4      9.5
     6
     7   6918689   17273675.0      2.5      8.7
     8   6792010   24472424.0      3.6     12.3
     9    174426     624074.0      3.6      0.3
    10
    11   6617584   24639431.0      3.7     12.4
    12         1          9.0      9.0      0.0

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [13]:
def mape_v1():
    pass


In [ ]:
def mape_v2():
    pass

In [ ]:
import numpy as np

def mape_v3(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [13]:
@numba.njit
def mape_v4(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4